In [10]:
import igl
import numpy as np
import scipy
from meshplot import plot, subplot, interact

mesh_dir = 'SHREC11/'

In [11]:
v, f = igl.read_triangle_mesh(mesh_dir + 'T0.obj')

In [12]:
f

array([[212, 221, 230],
       [231, 226, 230],
       [213, 227, 225],
       ...,
       [116,  95, 130],
       [116, 130, 133],
       [112, 131, 125]], dtype=int64)

In [37]:
distances = igl.all_pairs_distances(v,v,False)
adJ = igl.adjacency_matrix(f).toarray() * distances
diagonal = np.diag(np.sum(adJ,axis = 0))
lap = adJ - diagonal

In [42]:
lap

array([[-0.35945103,  0.07058634,  0.08883672, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.07058634, -0.46545782,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.08883672,  0.        , -0.43950675, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -0.54285612,
         0.07237107,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.07237107,
        -0.49586661,  0.13515637],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.13515637, -0.5232537 ]])